## Import modules

In [31]:
import os
import logging
from ipywidgets import Video
from multiprocessing import Pool
from itertools import repeat

from mdx.mtmc.core.calibration import Calibrator
from mdx.mtmc.core.data import Loader, Preprocessor
from mdx.mtmc.core.clustering import Clusterer
from mdx.mtmc.viz.mtmc_objects_in_grid import VizMTMCObjectsInGrid
from mdx.mtmc.config import AppConfig, VizMtmcConfig
from mdx.mtmc.utils.io_utils import load_json_from_file, make_clean_dir
from mdx.mtmc.utils.viz_mtmc_utils import hash_frames, hash_behaviors, hash_mtmc_objects
    
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%y/%m/%d %H:%M:%S", level=logging.INFO)

## Load configs and ground truth

In [32]:
app_config_path = "resources/app_mtmc_config.json"
assert os.path.exists(app_config_path), "App config not found"
app_config = AppConfig(**load_json_from_file(app_config_path))
app_config.io.enableDebug = True

viz_config_path = "resources/viz_mtmc_config.json"
assert os.path.exists(viz_config_path), "Viz config not found"
viz_config = VizMtmcConfig(**load_json_from_file(viz_config_path))
viz_config.setup.vizMtmcObjectsMode = "grid"
viz_config.setup.enableMultiprocessing = True
viz_config.setup.ffmpegRequired = True
viz_config.io.selectedGlobalIds = ["1"]
viz_config.io.outputDirPath = "results"
viz_config.plotting.gridLayout = [3, 3]
viz_config.plotting.outputFrameHeight = 1080

calibration_path = "resources/calibration_building_k.json"
assert os.path.exists(calibration_path), "Calibration info not found"

## Load raw data and process by batch processing

In [33]:
# Calibrate sensors
calibrator = Calibrator(app_config)
sensor_state_objects = calibrator.calibrate(calibration_path)

# Load JSON data from the perception pipeline
loader = Loader(app_config)
frames = None
json_data_path = app_config.io.jsonDataPath
protobuf_data_path = app_config.io.protobufDataPath
if os.path.isfile(json_data_path):
    frames = loader.load_json_data_to_frames(json_data_path)
elif os.path.isfile(protobuf_data_path):
    frames = loader.load_protobuf_data_to_frames(protobuf_data_path)
else:
    logging.error(f"ERROR: The JSON data path {json_data_path} and "
                  f"protobuf data path {protobuf_data_path} do NOT exist.")
    exit(1)

# Preprocess frames into behaviors and filter outliers
preprocessor = Preprocessor(app_config)
preprocessor.set_sensor_state_objects(sensor_state_objects)
behaviors = preprocessor.preprocess(frames)

# Cluster behaviors to get MTMC objects
clusterer = Clusterer(app_config)
mtmc_objects, _, _ = clusterer.cluster(behaviors)

25/01/21 17:58:25 - Calibrating sensor Building_K_Cam1...
25/01/21 17:58:25 - Calibrating sensor Building_K_Cam2...
25/01/21 17:58:25 - Calibrating sensor Building_K_Cam3...
25/01/21 17:58:25 - Calibrating sensor Building_K_Cam4...
25/01/21 17:58:25 - Calibrating sensor Building_K_Cam5...
25/01/21 17:58:25 - Calibrating sensor Building_K_Cam6...
25/01/21 17:58:25 - Calibrating sensor Building_K_Cam7...
25/01/21 17:58:25 - No. calibrated sensors: 7
25/01/21 17:58:25 - Loading JSON data from metropolis-apps-data/playback/mtmc_buildingK_playback.json...
25/01/21 17:58:51 - No. frames: 86476
25/01/21 17:58:51 - No. objects: 310085
25/01/21 17:58:58 - Filtering behaviors...
25/01/21 17:58:58 - No. behaviors filtered by confidence: 149
25/01/21 17:58:58 - No. behaviors filtered by bbox: 45
25/01/21 17:58:58 - No. behaviors filtered by behavior length: 0
25/01/21 17:58:58 - No. behaviors with empty embedding: 18
25/01/21 17:58:58 - No. behaviors after filtering: 413
25/01/21 17:58:58 - Findin

## Hash lists to lookup tables

In [34]:
frames = hash_frames(frames, viz_config.io.selectedSensorIds)
behaviors = hash_behaviors(behaviors, viz_config.io.selectedSensorIds,
                           viz_config.io.selectedBehaviorIds)
mtmc_objects = hash_mtmc_objects(mtmc_objects, viz_config.io.selectedGlobalIds)

## Visualize MTMC objects in grid mode

In [27]:
viz_mtmc_objects_in_grid = VizMTMCObjectsInGrid(viz_config)
output_video_dir = os.path.join(viz_config.io.outputDirPath, f"viz_{viz_config.setup.vizMode}_in_{viz_config.setup.vizMtmcObjectsMode}")
make_clean_dir(output_video_dir)
if not viz_config.setup.enableMultiprocessing:
    for global_id in mtmc_objects.keys():
        viz_mtmc_objects_in_grid.plot(global_id, output_video_dir, frames, behaviors, mtmc_objects)
else:
    with Pool() as pool:
        pool.starmap(viz_mtmc_objects_in_grid.plot,
                    zip(list(mtmc_objects), repeat(output_video_dir), repeat(frames),
                        repeat(behaviors), repeat(mtmc_objects)))

## Display output video

In [28]:
output_video_path = os.path.join(output_video_dir, f"object_id_{viz_config.io.selectedGlobalIds[0]}.mp4")
Video.from_file(output_video_path, width=480)

FileNotFoundError: [Errno 2] No such file or directory: 'results/viz_mtmc_objects_in_grid/object_id_1.mp4'